In [1]:
import pandas as pd
import numpy as np
import json
import google.generativeai as genai
from google.generativeai.types import HarmBlockThreshold, HarmCategory
import json

with open('/Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/config.txt', 'r') as config_file:
    config = json.load(config_file)
API_KEY = config["API_KEY"]
genai.configure(api_key=API_KEY)

/Users/lekshmanannatarajan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GEMINI_MODELS = {
    'FLASH':"gemini-1.5-flash-002"
    ,'PRO':"gemini-1.5-pro-002"
    ,'FLASH_8B':"gemini-1.5-flash-8b-001"
}

model_safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

generation_config = genai.GenerationConfig(
    max_output_tokens=3000,
    temperature=0.8,
    top_k=1,
    top_p=0.95,
)

In [ ]:
catagories_ = ['help me brainstorm', 'summarize this report for me', 'explain how to solve with steps', 'check if this was AI generated']

# Define the configuration with structured output
generation_config = genai.GenerationConfig(
    max_output_tokens=8000,
    temperature=0.7,
    top_k=1,
    top_p=0.95,
    response_mime_type="application/json",
    response_schema={
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "prompt": {"type": "string", "description": "Generated user prompt"},
                "truth_label": {"type": "string", "enum": f"{catagories_}"}
            },
            "required": ["prompt", "truth_label"]
        }
    }
)

# Create the model
gemini_model = genai.GenerativeModel(
    model_name=GEMINI_MODELS['PRO'],
    system_instruction=(
        "You are an AI agent trained to generate structured output"
        "Ensure the output adheres to the defined JSON schema."
    ),
    generation_config=generation_config,
    safety_settings=model_safety_settings
)

# Generate structured content
response = gemini_model.generate_content(
    contents=[f"""I need training/validation data to fine-tune a LLM classifier. Generate 10 prompts for each category in {catagories_} like how a student in college would ask"""])
    
# Print the JSON output
print(response.text)

In [8]:
pd.DataFrame(json.loads(response.text)).to_csv("./data_local/outputs/synthetic_test_prompts.csv", index=False)
pd.DataFrame(json.loads(response.text)).head(20)

,prompt,truth_label
0,I need help brainstorming ideas for my history...,help me brainstorm
1,Brainstorm with me: I'm making a short film an...,help me brainstorm
2,Help me brainstorm some fun activities for a c...,help me brainstorm
3,I'm starting a small business. Help me brains...,help me brainstorm
4,I'm writing a poem about nature. Can you help...,help me brainstorm
5,Help me brainstorm some persuasive arguments f...,help me brainstorm
6,I need to brainstorm some healthy and easy mea...,help me brainstorm
7,Can you help me brainstorm some creative ways ...,help me brainstorm
8,I'm stuck on this coding problem. Can you hel...,help me brainstorm
9,Help me brainstorm some potential research top...,help me brainstorm
